# Cosmic Ray Simulation

In [1]:
%matplotlib widget
from collections import defaultdict
import glob
import os
import sys

import astropy.constants as physical_constants
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.visualization import ImageNormalize, SqrtStretch, LogStretch, LinearStretch, ZScaleInterval, ManualInterval
import astropy.units as u
import dask.array as da
import h5py
import iminuit
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.style.use('ggplot')
import numpy as np
import pandas as pd
from pylandau import landau, langau
import scipy.ndimage as ndimage
from scipy.optimize import least_squares
from tqdm import tqdm



In [2]:
base_path = os.path.join(os.path.dirname(os.getcwd()))
pipeline_path = os.path.join(os.path.dirname(os.getcwd()), 'pipeline/')
sys.path.append(pipeline_path)

In [3]:
pipeline_path

'/ifs/missions/projects/plcosmic/hst_cosmic_rays/pipeline/'

In [4]:
from label import labeler as lbr
from stat_utils import statshandler as sh
from utils import datahandler as dh

In [5]:
detector_shape = {
    'STIS_CCD': (1024, 1024),
    'ACS_WFC': (4096, 4096),
    'WFPC2': (4*800, 800),
    'ACS_HRC': (1024, 1024),
    'WFC3_UVIS': (2051*2, 4096)
}

In [6]:
def reconstruct_cr_mask(cr_pixels, detector_shape):
    try:
        x_coords = np.array(cr_pixels)[:, 1].astype(np.int16)
    except IndexError as e:
        return
    y_coords = np.array(cr_pixels)[:, 0].astype(np.int16)
    array = np.zeros(shape=detector_shape, dtype=np.float32)
    array[y_coords, x_coords] = 1.
    label, num_sources = ndimage.label(array, structure=np.ones((3,3)))    
    return array, label, num_sources

In [7]:
def read_random_image(energy, cr_pixels, size, N=1, detector_shape=(1024, 1024)):
    flist_tuple = list(zip(energy.hdf5_files, cr_pixels.hdf5_files, size.hdf5_files))
    
    for (f1, f2, f3) in flist_tuple:
        fobj1 = h5py.File(f1, mode='r')
        grp1 = fobj1['/energy_deposited']
        
        fobj2 = h5py.File(f2, mode='r')
        grp2 = fobj2['/cr_affected_pixels']
        
        fobj3 = h5py.File(f3, mode='r')
        grp3 = fobj3['/sizes']
        
        for i, key in tqdm(enumerate(grp1.keys()), total=len(grp1.keys())):
      
            energy_dset = grp1[key]
            missing = False

            meta = energy_dset.attrs
            if meta['integration_time'] < 500:
                continue

In [8]:
def read_subset(energy, cr_pixels, size, N=1, detector_shape=(1024, 1024)):
    flist_tuple = list(zip(energy.hdf5_files, cr_pixels.hdf5_files, size.hdf5_files))
    dout = defaultdict(list)
    count = 0
    for (f1, f2, f3) in flist_tuple:
        fobj1 = h5py.File(f1, mode='r')
        grp1 = fobj1['/energy_deposited']
        
        fobj2 = h5py.File(f2, mode='r')
        grp2 = fobj2['/cr_affected_pixels']
        
        fobj3 = h5py.File(f3, mode='r')
        grp3 = fobj3['/sizes']
        
        for i, key in tqdm(enumerate(grp1.keys()), total=len(grp1.keys())):
            # Break the inner loop
            if i != N:
                continue
                
            energy_dset = grp1[key]
            missing = False

            meta = energy_dset.attrs
            if meta['integration_time'] < 500:
                continue

            try:
                cr_pixels = grp2[key]
            except IndexError as e:
                print(e)
                missing = True
          
            try:
                size_dset = grp3[key]
            except IndexError as e:
                print(e)
                missing = True
                

                
            if missing:
                continue
            
            crmask, label, num_sources = reconstruct_cr_mask(cr_pixels, detector_shape)
            dout['energy_deposited'] += list(energy_dset.value)
            dout['size_pix'] += list(size_dset[:][1])
            dout['cr_pixels'] += list(cr_pixels.value)
           
            if i > N: break
            
        break
        
    return dout, crmask, label, num_sources, meta

In [9]:
instr = 'ACS_WFC'
reader_energy = dh.DataReader(instr=instr, statistic='energy_deposited')
reader_energy.find_hdf5()

reader_size = dh.DataReader(instr=instr, statistic='sizes')
reader_size.find_hdf5()

reader_cr_pixels = dh.DataReader(instr=instr, statistic='cr_affected_pixels')
reader_cr_pixels.find_hdf5()

/ifs/missions/projects/plcosmic/hst_cosmic_rays/pipeline/utils/datahandler.py:220: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self._cfg = yaml.load(fobj)
INFO [datahandler.find_hdf5:301] Found the following data files
 /ifs/missions/projects/plcosmic/hst_cosmic_rays/results/ACS/acs_wfc_cr_energy_deposited_1.hdf5
/ifs/missions/projects/plcosmic/hst_cosmic_rays/results/ACS/acs_wfc_cr_energy_deposited_2.hdf5
/ifs/missions/projects/plcosmic/hst_cosmic_rays/results/ACS/acs_wfc_cr_energy_deposited_3.hdf5
/ifs/missions/projects/plcosmic/hst_cosmic_rays/results/ACS/acs_wfc_cr_energy_deposited_4.hdf5 
-------------------------------------------------------------------------------
INFO [datahandler.find_hdf5:301] Found the following data files
 /ifs/missions/projects/plcosmic/hst_cosmic_rays/results/ACS/acs_wfc_cr_sizes_1.hdf5
/ifs/missions/projects/plcosmic/hst_cosmic_rays/re

In [10]:
reader_energy.instr_cfg

{'search_pattern': '/data/ACS/WFC/mastDownload/HST/*/*flt.fits',
 'hdf5_files': {'cr_affected_pixels': '/results/ACS/acs_wfc_cr_affected_pixels.hdf5',
  'incident_cr_rate': '/results/ACS/acs_wfc_cr_rate.hdf5',
  'sizes': '/results/ACS/acs_wfc_cr_sizes.hdf5',
  'shapes': '/results/ACS/acs_wfc_cr_shapes.hdf5',
  'energy_deposited': '/results/ACS/acs_wfc_cr_energy_deposited.hdf5'},
 'failed': '/results/ACS/acs_wfc_failed_observations.txt',
 'astroquery': {'date_range': '2002-03-01',
  'SubGroupDescription': ['FLT', 'SPT'],
  'download_dir': '/data/ACS/WFC/'},
 'crrejtab': '/data/ACS/29p1548cj_crr_WFC.fits',
 'instr_params': {'extnums': [1, 2],
  'readout_time': 50.0,
  'gain_keyword': 'ATODGN*',
  'detector_size': 37.748,
  'pixel_size': 15}}

In [11]:
dout, crmask, label, num_sources, meta = read_subset(
    reader_energy, 
    reader_cr_pixels,
    reader_size,
    N=500,
    detector_shape=detector_shape[instr])

  0%|          | 0/3700 [00:00<?, ?it/s]/user/nmiles/miniconda3/envs/astroconda36/lib/python3.6/site-packages/ipykernel_launcher.py:45: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/user/nmiles/miniconda3/envs/astroconda36/lib/python3.6/site-packages/ipykernel_launcher.py:47: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
100%|██████████| 3700/3700 [00:00<00:00, 5186.24it/s]


In [12]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.imshow(crmask, interpolation='nearest',  origin='lower')
ax.grid(False)
ax.set_xlabel('X [pix]')
ax.set_ylabel('Y [pix]')
# ax.set_title('Reconstructed STIS/CCD CR map')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Y [pix]')

In [13]:
def add_cr_energy(crmask, label, dout):
    label_ids = np.unique(label.flatten())[1:]
    mask_out = crmask.astype(np.float32)
    for idx in tqdm(label_ids):
        npix = dout['size_pix'][idx-1]
        deposition = dout['energy_deposited'][idx-1]
        scaling = np.random.random(size=int(npix))
        scaling_norm = scaling/scaling.sum()
        mask_out[label==idx] = scaling_norm * deposition
    return mask_out

In [14]:
crmask_final = add_cr_energy(crmask, label, dout)

100%|██████████| 58643/58643 [12:22<00:00, 79.03it/s]


In [15]:
norm = ImageNormalize(crmask_final, stretch=LinearStretch(), vmin=0, vmax=100)

In [16]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.imshow(crmask_final, norm=norm,  origin='lower', cmap='gray')
ax.grid(False)
ax.set_xlabel('X [pix]')
ax.set_ylabel('Y [pix]')
# ax.set_title('Reconstructed STIS/CCD CR map')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Y [pix]')

In [45]:
1. *u.GeV / (938.27 * u.MeV).to('GeV')

<Quantity 1.0657913>

In [42]:
np.sqrt(1-(1/1.065**2))

0.344006856370373

In [46]:
numerator = 2 * (0.511 *u.MeV) * (2.065**2 - 1)
denominator = 1 + 2 * 2.065 * (0.511/938.27) +(0.511/938.27)**2

In [54]:
numerator/denominator

<Quantity 3.32855013 MeV>

In [55]:
np.sqrt((2.065**2 -1)/2.065**2)

0.8749233100364877

In [52]:
k = (100 *u.keV).to('MeV') / (numerator/denominator)

In [53]:
k

<Quantity 0.03004311>

In [ ]:
chip1 = crmask_final[:800,:]
chip2 = crmask_final[800:1600,:]
chip3 = crmask_final[1600:2400, :]
chip4 = crmask_final[2400:, :]

In [18]:
chip1 = crmask_final[:2051,:].copy()
chip2 = crmask_final[2051:,:].copy()

In [ ]:
chip1 = crmask_final

In [19]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.imshow(chip1, norm=norm,  origin='lower', cmap='gray')
ax.grid(False)
ax.set_xlabel('X [pix]')
ax.set_ylabel('Y [pix]')
# ax.set_title('Reconstructed STIS/CCD CR map')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Y [pix]')

In [20]:
hdr_keys = ['date','integration_time']

In [21]:
meta['date'], meta['integration_time']

('2002-07-28 12:42:08.000', 1050.0)

In [22]:
num_sources

58643

In [23]:
np.unique(label)[1:].size

58643

In [24]:
reader_energy.instr_cfg['instr_params']['detector_size']

37.748

In [25]:
reader_energy.instr_cfg['instr_params']

{'extnums': [1, 2],
 'readout_time': 50.0,
 'gain_keyword': 'ATODGN*',
 'detector_size': 37.748,
 'pixel_size': 15}

In [26]:
avg_cr_rate = {
    'WFPC2': 1.189,
    'WFC3_UVIS': 1.199,
    'ACS_WFC':1.165,
    'STIS_CCD':0.956,
    'ACS_HRC':1.013,
}

In [27]:
hdr = fits.Header()
hdr['instr'] = instr
hdr['inttime'] = (meta['integration_time'], 'total integration time')

hdr['exptime'] = (meta['integration_time'] - reader_energy.instr_cfg['instr_params']['readout_time'], 'commanded exposure time')
hdr['date'] = (meta['date'], 'date of observation')
hdr['num_cr'] = (num_sources, 'number of crs in template')
hdr['det_area'] = (reader_energy.instr_cfg['instr_params']['detector_size'],'physical det. size [cm^2]')
hdr['cr_flux'] = (avg_cr_rate[instr],'avg CR flux [CR/s/cm^2]')
hdr['cr_rate'] = (round(avg_cr_rate[instr]*reader_energy.instr_cfg['instr_params']['detector_size'],2), 'avg CR rate [CR/s]')

In [28]:
hdr

INSTR   = 'ACS_WFC '                                                            
INTTIME =               1050.0 / total integration time                         
EXPTIME =               1000.0 / commanded exposure time                        
DATE    = '2002-07-28 12:42:08.000' / date of observation                       
NUM_CR  =                58643 / number of crs in template                      
DET_AREA=               37.748 / physical det. size [cm^2]                      
CR_FLUX =                1.165 / avg CR flux [CR/s/cm^2]                        
CR_RATE =                43.98 / avg CR rate [CR/s]                             

In [ ]:
chips_to_process = [chip2, chip1]

In [ ]:
hdulist = fits.HDUList()
hdulist.append(fits.PrimaryHDU(header=hdr))
for chip in chips_to_process:
    hdulist.append(fits.ImageHDU(data=chip))

In [ ]:
instr.lower()

In [ ]:
# hdulist.writeto(f'{instr.lower()}_cr_template.fits', overwrite=True)

In [37]:
def trim_template(cr_template, hdr, exptime=100):
    expected_num_crs = np.ceil(exptime * hdr['cr_rate'])
    print(expected_num_crs)
    cr_mask = np.where(cr_template > 0 , 1, 0)
    label, num_crs = ndimage.label(cr_mask, structure=np.ones((3,3)))
    indices = np.unique(label)[1:]
    print(num_crs)
    num_to_remove = num_crs - expected_num_crs
    crs_to_remove = np.random.choice(indices, size=int(num_to_remove))
    for idx in tqdm(crs_to_remove):
        cr_template[label==idx] = 0
    return cr_template
    

In [38]:
chip1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [39]:
chip1_altered = alter_template(chip1.copy(), hdr, exptime=30)

1320.0


  0%|          | 19/27452 [00:00<02:24, 189.65it/s]

28772


100%|██████████| 27452/27452 [02:20<00:00, 195.12it/s]


In [36]:
fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=True)
ax[0].imshow(chip1, norm=norm,  origin='lower', cmap='gray')
ax[1].imshow(chip1_altered, norm=norm,  origin='lower', cmap='gray')
for a in ax:
    a.grid(False)
    a.set_xlabel('X [pix]')
    a.set_ylabel('Y [pix]')
# ax.set_title('Reconstructed STIS/CCD CR map')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …